In [5]:
import os
import sys
import glob
import datetime
import numpy as np
import pandas as pd
from pathlib import Path
from utils.utils import find_recent_file

print("numpy version:", np.__version__)
print("pandas version:", pd.__version__)
print("Python version:", sys.version)


# dir_log = '/tmp/'
dir_log = '.'

numpy version: 1.23.5
pandas version: 2.0.3
Python version: 3.8.10 (default, May 26 2023, 14:05:08) 
[GCC 9.4.0]


#### Load RTSP capture results

In [6]:
path_rtsp_capture = find_recent_file(file_prefix='rtsp_capture_tmp', dir_log=dir_log)
df_rtsp = pd.read_csv(path_rtsp_capture)
# df_rtsp.dropna(axis=0, how='any', inplace=True)
df_rtsp.columns = df_rtsp.columns.str.replace('.', '_')
print(df_rtsp.info())
df_rtsp

Find the latest_file : ./rtsp_capture_tmp.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23934 entries, 0 to 23933
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   frame_time_epoch        23934 non-null  object 
 1   rtcp_senderssrc         87 non-null     object 
 2   rtcp_timestamp_ntp_msw  87 non-null     float64
 3   rtcp_timestamp_ntp_lsw  87 non-null     float64
 4   rtcp_timestamp_rtp      87 non-null     float64
 5   rtp_ssrc                23846 non-null  object 
 6   udp_port                23933 non-null  float64
 7   rtp_timestamp           23846 non-null  float64
 8   rtp_marker              23846 non-null  float64
dtypes: float64(6), object(3)
memory usage: 1.6+ MB
None


,frame_time_epoch,rtcp_senderssrc,rtcp_timestamp_ntp_msw,rtcp_timestamp_ntp_lsw,rtcp_timestamp_rtp,rtp_ssrc,udp_port,rtp_timestamp,rtp_marker
0,1691977987.156260714,NaN,NaN,NaN,NaN,0xc3ba1d37,8000.0,3.042407e+09,1.0
1,1691977987.159705489,NaN,NaN,NaN,NaN,0x9e49cf7b,8000.0,2.980748e+09,1.0
2,1691977987.159907961,NaN,NaN,NaN,NaN,0xb4f7983d,8000.0,4.070519e+09,1.0
3,1691977987.168367398,NaN,NaN,NaN,NaN,0x8e109adc,8000.0,1.631582e+09,1.0
4,1691977987.168905531,NaN,NaN,NaN,NaN,0x574eae3e,8000.0,1.952597e+09,1.0
...,...,...,...,...,...,...,...,...,...
23929,1691978050.860097612,NaN,NaN,NaN,NaN,0xb4f7983d,8000.0,4.076240e+09,1.0
23930,1691978050.866726610,NaN,NaN,NaN,NaN,0x8e109adc,8000.0,1.637317e+09,1.0
23931,1691978050.866991036,NaN,NaN,NaN,NaN,0xe4a7a30f,8000.0,2.529527e+09,1.0
23932,1691978050.867415821,NaN,NaN,NaN,NaN,0x574eae3e,8000.0,1.958342e+09,1.0


#### Verify the number of valid streams IDs (sender and receiver SSRC IDs).

In [7]:
ssrc_rtcp = set(df_rtsp.rtcp_senderssrc.dropna().unique())
ssrc_rtp= set(df_rtsp.rtp_ssrc.dropna().unique())
ssrc_uniuque = set(ssrc_rtcp) & set(ssrc_rtp) 

print(f'ssrc_rtcp : {ssrc_rtcp}')
print(f'ssrc_rtp  : {ssrc_rtp }')
print(f'Number of valid streams : {len(ssrc_uniuque)}, id : {ssrc_uniuque}')
 

ssrc_rtcp : {'0xaa9e335e', '0xb4f7983d', '0x5eb47a89', '0xe4a7a30f', '0xc3ba1d37', '0x8e109adc', '0x9e49cf7b', '0xedd1ee59', '0xf32d99eb', '0x574eae3e', '0x2f9fffb3', '0x8e97e5b6'}
ssrc_rtp  : {'0xb4f7983d', '0xaa9e335e', '0x5eb47a89', '0xe4a7a30f', '0xc3ba1d37', '0x8e109adc', '0x9e49cf7b', '0xedd1ee59', '0xf32d99eb', '0x574eae3e', '0x2f9fffb3', '0x8e97e5b6'}
Number of valid streams : 12, id : {'0xb4f7983d', '0xaa9e335e', '0x5eb47a89', '0xe4a7a30f', '0xc3ba1d37', '0x8e109adc', '0x9e49cf7b', '0xedd1ee59', '0xf32d99eb', '0x574eae3e', '0x2f9fffb3', '0x8e97e5b6'}


In [8]:
print('RTCP Packets')
df_rtsp.query("rtcp_senderssrc.notna()")

RTCP Packets


,frame_time_epoch,rtcp_senderssrc,rtcp_timestamp_ntp_msw,rtcp_timestamp_ntp_lsw,rtcp_timestamp_rtp,rtp_ssrc,udp_port,rtp_timestamp,rtp_marker
360,1691977924.902784286,0xaa9e335e,3.900967e+09,895773798.0,3.064878e+09,NaN,8001.0,NaN,NaN
721,1691977925.902755623,0xedd1ee59,3.900967e+09,896103531.0,1.144476e+09,NaN,8001.0,NaN,NaN
1081,1691977926.902367204,0x574eae3e,3.900967e+09,895446429.0,1.947176e+09,NaN,8001.0,NaN,NaN
1444,1691977927.906771327,0x8e97e5b6,3.900967e+09,899843374.0,4.071519e+09,NaN,8001.0,NaN,NaN
3776,1691977934.384372627,0xc3ba1d37,3.900967e+09,377451472.0,3.037657e+09,NaN,8001.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
21348,1691978034.903034399,0xaa9e335e,3.900967e+09,895966111.0,3.074778e+09,NaN,8001.0,NaN,NaN
21709,1691978035.903142104,0xedd1ee59,3.900967e+09,896036799.0,1.154376e+09,NaN,8001.0,NaN,NaN
22069,1691978036.902297164,0x574eae3e,3.900967e+09,895171117.0,1.957076e+09,NaN,8001.0,NaN,NaN
22432,1691978037.906564012,0x8e97e5b6,3.900967e+09,899431644.0,4.081420e+09,NaN,8001.0,NaN,NaN


In [9]:
print('RTP Packets')
df_rtsp.query("rtcp_senderssrc.isna()")

RTP Packets


,frame_time_epoch,rtcp_senderssrc,rtcp_timestamp_ntp_msw,rtcp_timestamp_ntp_lsw,rtcp_timestamp_rtp,rtp_ssrc,udp_port,rtp_timestamp,rtp_marker
0,1691977987.156260714,NaN,NaN,NaN,NaN,0xc3ba1d37,8000.0,3.042407e+09,1.0
1,1691977987.159705489,NaN,NaN,NaN,NaN,0x9e49cf7b,8000.0,2.980748e+09,1.0
2,1691977987.159907961,NaN,NaN,NaN,NaN,0xb4f7983d,8000.0,4.070519e+09,1.0
3,1691977987.168367398,NaN,NaN,NaN,NaN,0x8e109adc,8000.0,1.631582e+09,1.0
4,1691977987.168905531,NaN,NaN,NaN,NaN,0x574eae3e,8000.0,1.952597e+09,1.0
...,...,...,...,...,...,...,...,...,...
23929,1691978050.860097612,NaN,NaN,NaN,NaN,0xb4f7983d,8000.0,4.076240e+09,1.0
23930,1691978050.866726610,NaN,NaN,NaN,NaN,0x8e109adc,8000.0,1.637317e+09,1.0
23931,1691978050.866991036,NaN,NaN,NaN,NaN,0xe4a7a30f,8000.0,2.529527e+09,1.0
23932,1691978050.867415821,NaN,NaN,NaN,NaN,0x574eae3e,8000.0,1.958342e+09,1.0
